## DATA

- source : [실거래가 공개 시스템](http://rtdown.molit.go.kr/)
- name : 아파트(매매) 실거래가 정보
- range : from 01-2006 to 02-2022


## CONTENT

1. merge all data into a file
2. remove columns named ['번지', '거래유형', '중개사소재지']
3. add columns named ['번지', '위도', '경도']

## USED PACKAGES
- pandas
- numpy
- os
- googlemaps

##### Before Start

In [2]:
import googlemaps
import numpy as np
import pandas as pd

from os import getcwd, listdir
from os.path import join

from typing import Optional

#### GET ALL DATA

In [3]:

# get data folder path
cur_path = getcwd()
data_path = join(cur_path,'data')

# get file names
names = listdir(data_path)

# get csv data by pandas in dataframe
years = np.arange(2006,2022).astype(str)
data = { y:pd.read_csv(join(data_path,n),sep=',',encoding='euc-kr') for y,n in zip(years,names)}

data.keys()

dict_keys(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'])

### 1. merge all data in a file

In [4]:
# merge
merged:Optional[pd.DataFrame] = pd.DataFrame({})
for i in data.values():
    merged = pd.concat([merged,i.copy()],axis=0)

# reset index
merged.index = np.arange(0,merged.shape[0])

#### refine data

In [5]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157918 entries, 0 to 1157917
Data columns (total 15 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   시군구       1157918 non-null  object 
 1   번지        1157700 non-null  object 
 2   본번        1157845 non-null  float64
 3   부번        1157845 non-null  float64
 4   단지명       1157918 non-null  object 
 5   전용면적(㎡)   1157918 non-null  float64
 6   계약년월      1157918 non-null  int64  
 7   계약일       1157918 non-null  int64  
 8   거래금액(만원)  1157918 non-null  object 
 9   층         1157918 non-null  int64  
 10  건축년도      1157918 non-null  int64  
 11  도로명       1157918 non-null  object 
 12  해제사유발생일   4400 non-null     float64
 13  거래유형      1157918 non-null  object 
 14  중개사소재지    1157918 non-null  object 
dtypes: float64(4), int64(4), object(7)
memory usage: 141.3+ MB


In [6]:
merged.describe()

,본번,부번,전용면적(㎡),계약년월,계약일,층,건축년도,해제사유발생일
count,1.157845e+06,1.157845e+06,1.157918e+06,1.157918e+06,1.157918e+06,1.157918e+06,1.157918e+06,4.400000e+03
mean,5.629858e+02,5.974497e+00,7.704780e+01,2.013773e+05,1.582013e+01,8.786981e+00,1.998311e+03,2.020499e+07
std,5.119306e+02,4.587584e+01,2.909836e+01,4.713187e+02,8.727215e+00,5.925182e+00,9.183139e+00,5.406550e+03
min,0.000000e+00,0.000000e+00,1.002000e+01,2.006010e+05,1.000000e+00,-4.000000e+00,1.961000e+03,2.020022e+07
25%,1.770000e+02,0.000000e+00,5.963000e+01,2.009080e+05,8.000000e+00,4.000000e+00,1.992000e+03,2.020072e+07
50%,4.700000e+02,0.000000e+00,8.116000e+01,2.015040e+05,1.600000e+01,8.000000e+00,1.999000e+03,2.020110e+07
75%,7.770000e+02,1.000000e+00,8.496000e+01,2.017120e+05,2.300000e+01,1.200000e+01,2.004000e+03,2.021042e+07
max,4.969000e+03,2.837000e+03,4.243200e+02,2.022020e+05,3.100000e+01,6.900000e+01,2.021000e+03,2.022022e+07


In [7]:
merged.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988,언주로 103,NaN,-,-
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988,언주로 103,NaN,-,-
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988,언주로 103,NaN,-,-
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,NaN,-,-
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,NaN,-,-


In [8]:
merged.tail()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
1157913,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202201,11,"85,000",16,2014,신내역로1길 85,NaN,중개거래,서울 중랑구
1157914,서울특별시 중랑구 중화동,208-4,208.0,4.0,범양프레체,17.94,202201,27,"12,200",15,2012,중랑천로14길 58,NaN,직거래,-
1157915,서울특별시 중랑구 중화동,208-4,208.0,4.0,범양프레체,17.94,202201,28,"11,400",15,2012,중랑천로14길 58,NaN,중개거래,서울 중랑구
1157916,서울특별시 중랑구 중화동,208-4,208.0,4.0,범양프레체,17.94,202202,10,"11,000",14,2012,중랑천로14길 58,NaN,중개거래,서울 중랑구
1157917,서울특별시 중랑구 중화동,453,453.0,0.0,청광플러스원,84.96,202201,11,"85,500",11,2007,동일로136길 15,NaN,중개거래,서울 중랑구


In [9]:
merged.isna().sum()

시군구               0
번지              218
본번               73
부번               73
단지명               0
전용면적(㎡)           0
계약년월              0
계약일               0
거래금액(만원)          0
층                 0
건축년도              0
도로명               0
해제사유발생일     1153518
거래유형              0
중개사소재지            0
dtype: int64

### <b>추가 조치사항 발생</b>
- 해제사유발생일이 NA인 Cell이 너무 많은 관계로 해당 컬럼 삭제
- 본번 및 부번이 없는 데이터 처리

#### 2. remove columns named ['번지', '거래유형', '중개사소재지']

In [10]:
dropped_columns = merged.drop(['번지','거래유형','중개사소재지'],axis=1)
dropped_columns.columns

Index(['시군구', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명', '해제사유발생일'],
      dtype='object')

### 2-2 추가 조치사항 진행

##### N1. 해제사유발생일이 NA인 Cell이 너무 많은 관계로 해당 컬럼 삭제

In [11]:
dropped_cancel = dropped_columns.drop(['해제사유발생일'],axis=1)
dropped_cancel.columns

Index(['시군구', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '층',
       '건축년도', '도로명'],
      dtype='object')

##### N2. 본번 및 부번이 없는 데이터 처리

In [12]:
fill_address = dropped_cancel.copy()

fill_address[(fill_address['본번'].isna()|fill_address['부번'].isna())]['단지명'].unique() # 힐스테이트 서초 젠트리스
fill_address.loc[(fill_address['본번'].isna()|fill_address['부번'].isna()) ,['본번','부번']].isna().sum() # 본번 73, 지번 73

# 본번 557, 부번 없음(0)
fill_address.loc[fill_address['본번'].isna()|fill_address['부번'].isna(),['본번']] = 557
fill_address.loc[fill_address['본번'].isna()|fill_address['부번'].isna(),['부번']] = 0

# check
fill_address[fill_address['본번'].isna()|fill_address['부번'].isna()].shape

(0, 11)

### 3. add columns named ['번지', '위도', '경도']

##### 3-1 새로운 '번지' 열 만들기

In [13]:
add_cols = fill_address.copy()

# 본번과 부번을 숫자로 바꾼다.
add_cols.loc[:,'본번'] = add_cols.loc[:,'본번'].astype(int)
add_cols.loc[:,'부번'] = add_cols.loc[:,'부번'].astype(int)

main = add_cols['본번'].to_numpy().astype(str)
sub = add_cols['부번'].to_numpy().astype(str)

joined_list = [ '-'.join([i,j]) if j !='0' else i for i,j in zip(main,sub) ]

new_column = pd.DataFrame({'번지':joined_list})
df_added_old = pd.concat([add_cols,new_column],axis=1)

df_added_old.head(3)

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,번지
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988,언주로 103,655-2
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988,언주로 103,655-2
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988,언주로 103,655-2


##### 3-2 도로명 주소로부터 위도와 경도 구하기

In [57]:
import env
maps = googlemaps.Client(key=env.KEY)  # my key값 입력

df_lat_lng = df_added_old.copy()
address = pd.Series(name='주소',data = list(map(lambda x: ' '.join(x),df_lat_lng[['시군구','도로명']].to_numpy())))
lat,lng = [],[]

for i in address.unique():
    try:
        location = maps.geocode(i)[0].get('geometry')
        print(location['location']['lat'], location['location']['lng'])
        lat.append(location['location']['lat'])
        lng.append(location['location']['lng'])
    except Exception as e:
        print(e)
        print('ERROR : ',i)

37.4813054 127.052897
37.4762642 127.0567424
37.4839447 127.0561699
37.4960876 127.0767362
37.4829571 127.0589875
37.478368 127.0614839
37.4841132 127.067294
37.48553709999999 127.0714351
37.4875354 127.0686558
37.48883259999999 127.071487
37.4816442 127.0526666
37.4797123 127.0459467
37.4843872 127.054267
37.4747735 127.0547895
37.47435189999999 127.0548027
37.49396540000001 127.0750977
37.4871869 127.0602688
37.473145 127.0529491
37.4732407 127.0530892
37.4733275 127.0532351
37.49396540000001 127.0750977
37.4785609 127.0542624
37.4727746 127.0523738
37.4828707 127.0612704
37.4881707 127.0625135
37.4848024 127.0571779
37.4793154 127.0489038
37.4794774 127.0487672
37.4795126 127.0491105
37.4831816 127.0618659
37.4745812 127.0551887
37.4740704 127.0537435
37.4739021 127.0541063
37.4842818 127.0601356
37.4825481 127.0544891
37.482369 127.0513673
37.4878994 127.0609236
37.5066597 127.0289875
37.5062954 127.0276384
37.5106451 127.0381352
37.5135304 127.0315341
37.5135304 127.0315341
37.508

In [60]:
pos = pd.DataFrame({'위도':lat, '경도':lng,'주소':address.unique().tolist()})
pos.to_csv('./processed/pos.csv')
add_address = pd.concat([df_lat_lng.copy(),address],axis=1)
final = add_address.merge(pos,on='주소',how='left',suffixes=('_',''))

# check data
final.to_csv('./processed/preprocessed.csv')


## 점검

In [62]:
final.isna().sum()

시군구         0
본번          0
부번          0
단지명         0
전용면적(㎡)     0
계약년월        0
계약일         0
거래금액(만원)    0
층           0
건축년도        0
도로명         0
번지          0
주소          0
위도          0
경도          0
dtype: int64

In [63]:
final.head()

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,번지,주소,위도,경도
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200603,10,"59,500",7,1988,언주로 103,655-2,서울특별시 강남구 개포동 언주로 103,37.481305,127.052897
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200603,29,"60,000",6,1988,언주로 103,655-2,서울특별시 강남구 개포동 언주로 103,37.481305,127.052897
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200604,29,"67,000",9,1988,언주로 103,655-2,서울특별시 강남구 개포동 언주로 103,37.481305,127.052897
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,655-2,서울특별시 강남구 개포동 언주로 103,37.481305,127.052897
4,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,655-2,서울특별시 강남구 개포동 언주로 103,37.481305,127.052897


In [64]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157918 entries, 0 to 1157917
Data columns (total 15 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   시군구       1157918 non-null  object 
 1   본번        1157918 non-null  int32  
 2   부번        1157918 non-null  int32  
 3   단지명       1157918 non-null  object 
 4   전용면적(㎡)   1157918 non-null  float64
 5   계약년월      1157918 non-null  int64  
 6   계약일       1157918 non-null  int64  
 7   거래금액(만원)  1157918 non-null  object 
 8   층         1157918 non-null  int64  
 9   건축년도      1157918 non-null  int64  
 10  도로명       1157918 non-null  object 
 11  번지        1157918 non-null  object 
 12  주소        1157918 non-null  object 
 13  위도        1157918 non-null  float64
 14  경도        1157918 non-null  float64
dtypes: float64(3), int32(2), int64(4), object(6)
memory usage: 132.5+ MB
